In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}


url = 'https://www.healthcollect.vic.gov.au/HospitalLists/MainHospitalList.aspx'
response = requests.get(url, headers=header)
print(response.status_code)

soup = BeautifulSoup(response.text, 'html.parser')



table_element = soup.find('table', {"id": "x:201238496.4:mkr:dataTbl.hdn"})  
print(table_element)

data_list = []

for row in table_element.find_all('tr'):

    columns = row.find_all('td')
    

    if len(columns) >= 3:
       
        address = columns[3].get_text(strip=True)
        suburb = columns[4].get_text(strip=True)
        combined_data = address + ' ' + suburb
        data_list.append([combined_data])

#for data in column_4_data:
    #print(data)

with open('../data/raw/hospital_address.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    #writer.writerow(['Address', 'Suburb'])  
    writer.writerows(data_list)

200
<table border="0" cellpadding="0" cellspacing="0" id="x:201238496.4:mkr:dataTbl.hdn" style="table-layout:fixed;width:100%;visibility:hidden"><tbody class="ig_Item igg_Item" id="x:201238496.12:mkr:rows:nw:1"><tr id="x:201238496.13:adr:0:tag:"><td>Albert Road Clinic</td><td>Albert Road Clinic</td><td style="width:8%;">NO</td><td style="width:13%;">31-33 Albert Road</td><td style="width:10%;">South Melbourne</td><td style="width:6%;">3205</td><td style="width:5%;"></td><td style="width:7%;">PRIVATE</td><td style="width:12%;">Private Hospital</td></tr><!--["Albert Road Clinic"]--><tr class="ig_Alt igg_Alt" id="x:201238496.13:adr:1:tag:"><td>Albury Wodonga Health</td><td>Albury Wodonga Health</td><td>YES</td><td>69 Vermont Street</td><td>Wodonga</td><td>3690</td><td></td><td>PUBLIC</td><td>Public Hospital</td></tr><!--["Albury Wodonga Health"]--><tr id="x:201238496.13:adr:2:tag:"><td>Albury Wodonga Health, Albury Campus</td><td>Albury Wodonga Health, Albury Campus</td><td>YES</td><td>Bo

In [2]:

import requests
from geopy.exc import GeocoderTimedOut

def get_lat_lon(address):
# this function is used to get the longtitude and latitude of a specific address
# address: this is the input address
# return  : (latitude of this address, longtitude of this address)

    headers = {
        'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
    }
    
    url = f"https://nominatim.openstreetmap.org/search?q={address}&format=json"
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        data = response.json()
        
        if data and isinstance(data, list) and len(data) > 0:
            latitude = float(data[0]['lat'])
            longitude = float(data[0]['lon'])
            return latitude, longitude
        else:
            return None
    except (requests.Timeout, requests.RequestException):
        print("Geocoding request failed.")
        return None

with open('../data/raw/hospital_address.csv', 'r', encoding='utf-8') as input_csv, open('../data/raw/hospital_latlongitude.csv', 'w', newline='', encoding='utf-8') as output_csv:
    csv_reader = csv.reader(input_csv)
    #next(csv_reader)  
    csv_writer = csv.writer(output_csv)
    csv_writer.writerow(['Address', 'Latitude', 'Longitude'])  

    for row in csv_reader:
        address = row[0] 
        coordinates = get_lat_lon(address)
        if coordinates:
            latitude, longitude = coordinates
            csv_writer.writerow([address, latitude, longitude])
        else:
            csv_writer.writerow([address, '', 'Geocoding failed'])
            